In [2]:
import os
from getpass import getpass

# Set up API key
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

print("✅ Authentication configured!")

✅ Authentication configured!


In [3]:
#importing libs 
from typing import Any, Dict
import subprocess
import time
import requests
import json
import uuid

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.runners import InMemoryRunner
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a

print("✅ ADK components imported successfully (with A2A support).")


✅ ADK components imported successfully (with A2A support).


In [4]:
# Define helper function for A2A communication with session memory

# Global session ID for persistent memory across test queries
PERSISTENT_SESSION_ID = None
SESSION_INITIALIZED = False

async def test_a2a_query(user_query: str, use_persistent_session: bool = True):
    """
    Test the A2A communication between Root Agent and specialist agents.
    
    This function:
    1. Reuses the same session for conversation continuity (if use_persistent_session=True)
    2. Sends the query to the Root Agent
    3. Root Agent communicates with specialist agents via A2A
    4. Displays the response
    
    Args:
        user_query: The question to ask the Financial Assistant
        use_persistent_session: If True, reuses same session for memory. If False, creates new session.
    """
    global PERSISTENT_SESSION_ID, SESSION_INITIALIZED
    
    if use_persistent_session:
        # Initialize persistent session on first use
        if not SESSION_INITIALIZED:
            PERSISTENT_SESSION_ID = f"a2a_persistent_session_{uuid.uuid4().hex[:8]}"
            session = await session_service.create_session(
                app_name=APP_NAME, user_id=USER_ID, session_id=PERSISTENT_SESSION_ID
            )
            SESSION_INITIALIZED = True
        session_id = PERSISTENT_SESSION_ID
    else:
        # Create unique session ID for isolated test
        session_id = f"a2a_session_{uuid.uuid4().hex[:8]}"
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    
    # Create the user message
    test_content = types.Content(parts=[types.Part(text=user_query)])
    
    # Display query
    print(f"\n👤 User: {user_query}")
    print(f"\n🤖 Financial Assistant response:")
    print("-" * 80)
    
    # Run the agent asynchronously (handles streaming responses and A2A communication)
    async for event in runner.run_async(
        user_id=USER_ID, session_id=session_id, new_message=test_content
    ):
        # Print final response only (skip intermediate events)
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)
    
    print("-" * 80)


async def reset_session():
    """Reset the persistent session to start a fresh conversation."""
    global PERSISTENT_SESSION_ID, SESSION_INITIALIZED
    PERSISTENT_SESSION_ID = f"a2a_persistent_session_{uuid.uuid4().hex[:8]}"
    
    # Create the new session
    session = await session_service.create_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=PERSISTENT_SESSION_ID
    )
    SESSION_INITIALIZED = True
    
    print(f"✅ Session reset! New session ID: {PERSISTENT_SESSION_ID}")


print("✅ A2A test function defined with session memory support.")
print("💡 Use test_a2a_query(query) for persistent memory, or test_a2a_query(query, use_persistent_session=False) for isolated tests.")
print("💡 Use await reset_session() to start a fresh conversation.")

✅ A2A test function defined with session memory support.
💡 Use test_a2a_query(query) for persistent memory, or test_a2a_query(query, use_persistent_session=False) for isolated tests.
💡 Use await reset_session() to start a fresh conversation.


In [5]:
def calculate_pe_ratio(stock_price: float, earnings_per_share: float) -> dict:
    """Calculate Price-to-Earnings ratio."""
    if earnings_per_share == 0:
        return {"error": "EPS cannot be zero"}
    
    pe_ratio = stock_price / earnings_per_share
    return {
        "pe_ratio": round(pe_ratio, 2),
        "stock_price": stock_price,
        "eps": earnings_per_share
    }

def calculate_return(initial_value: float, final_value: float) -> dict:
    """Calculate investment return percentage."""
    if initial_value == 0:
        return {"error": "Initial value cannot be zero"}
    
    return_pct = ((final_value - initial_value) / initial_value) * 100
    return {
        "return_percentage": round(return_pct, 2),
        "initial_value": initial_value,
        "final_value": final_value,
        "profit_loss": round(final_value - initial_value, 2)
    }

def analyze_portfolio(holdings: dict) -> dict:
    """Analyze portfolio allocation."""
    total = sum(holdings.values())
    if total == 0:
        return {"error": "Portfolio value cannot be zero"}
    
    return {
        "total_value": total,
        "holdings": {
            symbol: {
                "value": value,
                "percentage": round((value/total)*100, 2)
            }
            for symbol, value in holdings.items()
        }
    }

print("✅ Financial tools ready!")

# Test the functions
print("\n📊 Testing tools:")
print("\n1. P/E Ratio Test:")
print(calculate_pe_ratio(150.0, 6.0))

print("\n2. Return Test:")
print(calculate_return(100.0, 125.0))

print("\n3. Portfolio Analysis Test:")
print(analyze_portfolio({"AAPL": 10000, "MSFT": 8000, "GOOGL": 12000}))


✅ Financial tools ready!

📊 Testing tools:

1. P/E Ratio Test:
{'pe_ratio': 25.0, 'stock_price': 150.0, 'eps': 6.0}

2. Return Test:
{'return_percentage': 25.0, 'initial_value': 100.0, 'final_value': 125.0, 'profit_loss': 25.0}

3. Portfolio Analysis Test:
{'total_value': 30000, 'holdings': {'AAPL': {'value': 10000, 'percentage': 33.33}, 'MSFT': {'value': 8000, 'percentage': 26.67}, 'GOOGL': {'value': 12000, 'percentage': 40.0}}}


In [6]:
# Watchlist Management Tools

# Define USER_ID early for use in tool functions
USER_ID = "default"

class WatchlistManager:
    def __init__(self):
        self.watchlists = {}  # {user_id: {symbol: note}}
    
    def add_to_watchlist(self, user_id: str, symbol: str, note: str = "") -> dict:
        """Add a stock to user's watchlist."""
        if user_id not in self.watchlists:
            self.watchlists[user_id] = {}
        
        symbol = symbol.upper()
        self.watchlists[user_id][symbol] = note
        
        return {
            "status": "success",
            "action": "added",
            "symbol": symbol,
            "note": note,
            "watchlist_size": len(self.watchlists[user_id])
        }
    
    def remove_from_watchlist(self, user_id: str, symbol: str) -> dict:
        """Remove a stock from user's watchlist."""
        symbol = symbol.upper()
        
        if user_id in self.watchlists and symbol in self.watchlists[user_id]:
            del self.watchlists[user_id][symbol]
            return {
                "status": "success",
                "action": "removed",
                "symbol": symbol,
                "watchlist_size": len(self.watchlists[user_id])
            }
        
        return {
            "status": "error",
            "message": f"{symbol} not found in watchlist"
        }
    
    def get_watchlist(self, user_id: str) -> dict:
        """Get user's entire watchlist."""
        if user_id not in self.watchlists:
            return {
                "status": "success",
                "watchlist": {},
                "count": 0
            }
        
        return {
            "status": "success",
            "watchlist": self.watchlists[user_id],
            "count": len(self.watchlists[user_id])
        }

# Initialize global watchlist manager
watchlist_manager = WatchlistManager()

# Create tool functions that use the manager
def add_to_watchlist(symbol: str, note: str = "") -> dict:
    """Add a stock to your watchlist with optional notes."""
    return watchlist_manager.add_to_watchlist(USER_ID, symbol, note)

def remove_from_watchlist(symbol: str) -> dict:
    """Remove a stock from your watchlist."""
    return watchlist_manager.remove_from_watchlist(USER_ID, symbol)

def get_my_watchlist() -> dict:
    """Get all stocks in your watchlist."""
    return watchlist_manager.get_watchlist(USER_ID)

print("✅ Watchlist management tools ready!")
print(f"📋 Test: {add_to_watchlist('AAPL', 'Interested in iPhone 16 sales')}")
print(f"📋 Test: {get_my_watchlist()}")


✅ Watchlist management tools ready!
📋 Test: {'status': 'success', 'action': 'added', 'symbol': 'AAPL', 'note': 'Interested in iPhone 16 sales', 'watchlist_size': 1}
📋 Test: {'status': 'success', 'watchlist': {'AAPL': 'Interested in iPhone 16 sales'}, 'count': 1}


# note

here we are defining the server codes for the agent, to run the servers it's better to run them on terminal 
1- calculator server:

`cd /tmp && uvicorn calculator_server:app --host localhost --port 8010 --reload`

2- watchlist server 

`cd /tmp && uvicorn watchlist_server:app --host localhost --port 8002 --reload`

3- google search server 

`cd /tmp && uvicorn search_server:app --host localhost --port 8003 --reload`



In [8]:
APP_NAME = "financial_assistant"
USER_ID = "default"
MODEL_NAME = "gemini-2.0-flash-lite"

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# 📝 Save specialist agents as separate server files for A2A deployment

# 1. Calculator Agent Server
calculator_agent_code = f'''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

# Set API key from environment
os.environ["GOOGLE_API_KEY"] = "{os.environ.get('GOOGLE_API_KEY')}"
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

def calculate_pe_ratio(stock_price: float, earnings_per_share: float) -> dict:
    """Calculate Price-to-Earnings ratio."""
    if earnings_per_share == 0:
        return {{"error": "EPS cannot be zero"}}
    
    pe_ratio = stock_price / earnings_per_share
    return {{
        "pe_ratio": round(pe_ratio, 2),
        "stock_price": stock_price,
        "eps": earnings_per_share
    }}

def calculate_return(initial_value: float, final_value: float) -> dict:
    """Calculate investment return percentage."""
    if initial_value == 0:
        return {{"error": "Initial value cannot be zero"}}
    
    return_pct = ((final_value - initial_value) / initial_value) * 100
    return {{
        "return_percentage": round(return_pct, 2),
        "initial_value": initial_value,
        "final_value": final_value,
        "profit_loss": round(final_value - initial_value, 2)
    }}

def analyze_portfolio(holdings: dict) -> dict:
    """Analyze portfolio allocation."""
    total = sum(holdings.values())
    if total == 0:
        return {{"error": "Portfolio value cannot be zero"}}
    
    return {{
        "total_value": total,
        "holdings": {{
            symbol: {{
                "value": value,
                "percentage": round((value/total)*100, 2)
            }}
            for symbol, value in holdings.items()
        }}
    }}

calculator_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="calculator_agent",
    description="Financial calculator that performs P/E ratios, returns, and portfolio analysis.",
    instruction="""You are a financial calculator specialist.
    
    Your job is to:
    - Calculate P/E ratios when given stock price and EPS
    - Calculate investment returns and profit/loss
    - Analyze portfolio allocations
    
    Always show your calculations clearly and explain the results.""",
    tools=[calculate_pe_ratio, calculate_return, analyze_portfolio],
)

app = to_a2a(calculator_agent, port=8010)
'''

# 2. Watchlist Agent Server
watchlist_agent_code = f'''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

os.environ["GOOGLE_API_KEY"] = "{os.environ.get('GOOGLE_API_KEY')}"
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

class WatchlistManager:
    def __init__(self):
        self.watchlists = {{}}
    
    def add_to_watchlist(self, user_id: str, symbol: str, note: str = "") -> dict:
        if user_id not in self.watchlists:
            self.watchlists[user_id] = {{}}
        symbol = symbol.upper()
        self.watchlists[user_id][symbol] = note
        return {{
            "status": "success",
            "action": "added",
            "symbol": symbol,
            "note": note,
            "watchlist_size": len(self.watchlists[user_id])
        }}
    
    def remove_from_watchlist(self, user_id: str, symbol: str) -> dict:
        symbol = symbol.upper()
        if user_id in self.watchlists and symbol in self.watchlists[user_id]:
            del self.watchlists[user_id][symbol]
            return {{
                "status": "success",
                "action": "removed",
                "symbol": symbol,
                "watchlist_size": len(self.watchlists[user_id])
            }}
        return {{"status": "error", "message": f"{{symbol}} not found in watchlist"}}
    
    def get_watchlist(self, user_id: str) -> dict:
        if user_id not in self.watchlists:
            return {{"status": "success", "watchlist": {{}}, "count": 0}}
        return {{
            "status": "success",
            "watchlist": self.watchlists[user_id],
            "count": len(self.watchlists[user_id])
        }}

watchlist_manager = WatchlistManager()

def add_to_watchlist(symbol: str, note: str = "") -> dict:
    """Add a stock to your watchlist with optional notes."""
    return watchlist_manager.add_to_watchlist("default", symbol, note)

def remove_from_watchlist(symbol: str) -> dict:
    """Remove a stock from your watchlist."""
    return watchlist_manager.remove_from_watchlist("default", symbol)

def get_my_watchlist() -> dict:
    """Get all stocks in your watchlist."""
    return watchlist_manager.get_watchlist("default")

watchlist_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="watchlist_agent",
    description="Watchlist manager that tracks stocks of interest.",
    instruction="""You are a watchlist management specialist.
    
    Your job is to:
    - Add stocks to the user's watchlist with notes
    - Remove stocks from the watchlist
    - Show the current watchlist
    
    When adding stocks, encourage users to add notes about why they're interested.""",
    tools=[add_to_watchlist, remove_from_watchlist, get_my_watchlist],
)

app = to_a2a(watchlist_agent, port=8002)
'''

# 3. Search Agent Server
search_agent_code = f'''
import os
from google.adk.agents import Agent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search
from google.genai import types

os.environ["GOOGLE_API_KEY"] = "{os.environ.get('GOOGLE_API_KEY')}"
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

search_agent = Agent(
    model=Gemini(model="gemini-2.0-flash-lite", retry_options=retry_config),
    name="search_agent",
    description="Market research specialist that searches for current stock information.",
    instruction="""You are a market research specialist.
    
    Your job is to:
    - Search for current stock prices and news
    - Find company financial data and earnings reports
    - Research market trends and analysis
    
    Always provide sources and recent information.""",
    tools=[google_search],
)

app = to_a2a(search_agent, port=8003)
'''

# Write agent server files
with open("/tmp/calculator_server.py", "w") as f:
    f.write(calculator_agent_code)

with open("/tmp/watchlist_server.py", "w") as f:
    f.write(watchlist_agent_code)

with open("/tmp/search_server.py", "w") as f:
    f.write(search_agent_code)

print("✅ Agent server files created!")
print("   📊 /tmp/calculator_server.py")
print("   📋 /tmp/watchlist_server.py")
print("   🔍 /tmp/search_server.py")


✅ Agent server files created!
   📊 /tmp/calculator_server.py
   📋 /tmp/watchlist_server.py
   🔍 /tmp/search_server.py


In [ ]:
#better to run on terminal, not here 
#  Start A2A Agent Servers (Non-blocking)

def start_a2a_server_background(module_name: str, port: int, agent_name: str):
    """Start a uvicorn server in background (non-blocking)."""
    process = subprocess.Popen(
        ["uvicorn", f"{module_name}:app", "--host", "localhost", "--port", str(port)],
        cwd="/tmp",
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        env={**os.environ},
    )
    
    print(f"🚀 Starting {agent_name} server on port {port} (background)...")
    
    # Brief wait to check if server starts (non-blocking)
    time.sleep(2)
    
    # Quick check if server is responding
    try:
        response = requests.get(
            f"http://localhost:{port}/.well-known/agent-card.json", timeout=2
        )
        if response.status_code == 200:
            print(f"✅ {agent_name} is running!")
            print(f"   URL: http://localhost:{port}\n")
        else:
            print(f"⏳ {agent_name} starting... (may take a few more seconds)\n")
    except requests.exceptions.RequestException:
        print(f"⏳ {agent_name} starting... (may take a few more seconds)\n")
    
    return process

# Start all three agent servers in background
print("🔧 Starting A2A agent servers in background...\n")

calculator_process = start_a2a_server_background("calculator_server", 8001, "Calculator Agent")
watchlist_process = start_a2a_server_background("watchlist_server", 8002, "Watchlist Agent")
search_process = start_a2a_server_background("search_server", 8003, "Search Agent")

# Store processes globally so we can stop them later
globals()["calculator_server_process"] = calculator_process
globals()["watchlist_server_process"] = watchlist_process
globals()["search_server_process"] = search_process

print("✅ All A2A agent servers started in background!")
print("⏳ Wait 5-10 seconds for all servers to be fully ready, then run the next cell.")
print("💡 Tip: If servers aren't ready, re-run cell 9 to check agent cards.")


🔧 Starting A2A agent servers in background...

🚀 Starting Calculator Agent server on port 8001 (background)...
⏳ Calculator Agent starting... (may take a few more seconds)

🚀 Starting Watchlist Agent server on port 8002 (background)...
⏳ Watchlist Agent starting... (may take a few more seconds)

🚀 Starting Search Agent server on port 8003 (background)...
⏳ Search Agent starting... (may take a few more seconds)

✅ All A2A agent servers started in background!
⏳ Wait 5-10 seconds for all servers to be fully ready, then run the next cell.
💡 Tip: If servers aren't ready, re-run cell 9 to check agent cards.


In [9]:
# 📋 Fetch and display agent cards from running servers

def fetch_agent_card(port: int, agent_name: str):
    """Fetch and display the agent card from a running A2A server."""
    try:
        response = requests.get(
            f"http://localhost:{port}/.well-known/agent-card.json", timeout=5
        )
        
        if response.status_code == 200:
            agent_card = response.json()
            print(f"📋 {agent_name} Card:")
            print(json.dumps(agent_card, indent=2))
            print(f"\n✨ Key Information:")
            print(f"   Name: {agent_card.get('name')}")
            print(f"   Description: {agent_card.get('description')}")
            print(f"   URL: {agent_card.get('url')}")
            print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
            print("\n" + "-" * 80 + "\n")
            return agent_card
        else:
            print(f"❌ Failed to fetch {agent_name} card: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {agent_name} card: {e}")
    return None

# Fetch all agent cards
calculator_card = fetch_agent_card(8010, "Calculator Agent")
watchlist_card = fetch_agent_card(8002, "Watchlist Agent")
search_card = fetch_agent_card(8003, "Search Agent")


📋 Calculator Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "Financial calculator that performs P/E ratios, returns, and portfolio analysis.",
  "name": "calculator_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "Financial calculator that performs P/E ratios, returns, and portfolio analysis. I am a financial calculator specialist.\n    \n    my job is to:\n    - Calculate P/E ratios when given stock price and EPS\n    - Calculate investment returns and profit/loss\n    - Analyze portfolio allocations\n    \n    Always show my calculations clearly and explain the results.",
      "id": "calculator_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Calculate Price-to-Earnings ratio.",
      "id": "calculator_agent-calculate_pe_ratio",
      "name": "calculate_pe_ratio",


In [10]:
# Create RemoteA2aAgent proxies for each specialist agent

# These act as client-side proxies - the root agent can use them like local agents
# but they communicate via A2A protocol with the remote servers

remote_calculator_agent = RemoteA2aAgent(
    name="calculator_agent",
    description="Remote financial calculator that performs P/E ratios, returns, and portfolio analysis.",
    agent_card=f"http://localhost:8010{AGENT_CARD_WELL_KNOWN_PATH}",
)

remote_watchlist_agent = RemoteA2aAgent(
    name="watchlist_agent",
    description="Remote watchlist manager that tracks stocks of interest.",
    agent_card=f"http://localhost:8002{AGENT_CARD_WELL_KNOWN_PATH}",
)

remote_search_agent = RemoteA2aAgent(
    name="search_agent",
    description="Remote market research specialist that searches for current stock information.",
    agent_card=f"http://localhost:8003{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote A2A Agent proxies created!")
print(f"   📊 Calculator Agent: Connected to http://localhost:8010")
print(f"   📋 Watchlist Agent: Connected to http://localhost:8002")
print(f"   🔍 Search Agent: Connected to http://localhost:8003")
print("\n   The root agent can now use these like local sub-agents via A2A protocol!")


✅ Remote A2A Agent proxies created!
   📊 Calculator Agent: Connected to http://localhost:8010
   📋 Watchlist Agent: Connected to http://localhost:8002
   🔍 Search Agent: Connected to http://localhost:8003

   The root agent can now use these like local sub-agents via A2A protocol!


/var/folders/vk/pp56pdfs6k9dv7pnhpg0x7980000gn/T/ipykernel_15069/3549118406.py:6: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_calculator_agent = RemoteA2aAgent(
/var/folders/vk/pp56pdfs6k9dv7pnhpg0x7980000gn/T/ipykernel_15069/3549118406.py:12: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_watchlist_agent = RemoteA2aAgent(
/var/folders/vk/pp56pd

In [11]:
#  Create Root Coordinator Agent that uses RemoteA2aAgents

root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="financial_assistant",
    description="Comprehensive financial assistant coordinating research, calculations, and watchlist.",
    instruction="""You are a comprehensive financial assistant that helps users with stock research and investment analysis.
    
    IMPORTANT: Always respond in English, regardless of the user's location or native language.
    
    You coordinate three specialist sub-agents via A2A protocol:
    1. **calculator_agent** - Use this for P/E ratios, returns, portfolio analysis
    2. **watchlist_agent** - Use this to add/remove/view stocks in the watchlist
    3. **search_agent** - Use this for current stock prices, news, company data, market research
    
    When a user asks about stocks:
    - Delegate to search_agent for current market information
    - Delegate to calculator_agent for financial calculations
    - Delegate to watchlist_agent for watchlist management
    - Combine insights from agents when needed
    - Suggest adding interesting stocks to the watchlist
    
    Always use the appropriate sub-agent for each task.
    Remember context from previous messages in the conversation.""",
    sub_agents=[remote_calculator_agent, remote_watchlist_agent, remote_search_agent],
)

# Session Management
session_service = InMemorySessionService()

# Create Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Financial Assistant with A2A Protocol initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Root Agent: {root_agent.name}")
print(f"   - Sub-agents: 3 remote agents via A2A")
print(f"   - Session Service: {session_service.__class__.__name__}")
print("\n🌐 A2A Architecture:")
print("   Root Agent → RemoteA2aAgent proxies → A2A servers (ports 8010, 8002, 8003)")
print("\n✨ Language: Always responds in English")

✅ Financial Assistant with A2A Protocol initialized!
   - Application: financial_assistant
   - User: default
   - Root Agent: financial_assistant
   - Sub-agents: 3 remote agents via A2A
   - Session Service: InMemorySessionService

🌐 A2A Architecture:
   Root Agent → RemoteA2aAgent proxies → A2A servers (ports 8010, 8002, 8003)

✨ Language: Always responds in English


## Multi-Agent Financial Assistant

Creating a system with:
1. **Search Agent** - Handles Google Search for stock news and prices
2. **Calculator Agent** - Handles financial calculations
3. **Root Agent** - Coordinates between the two agents

## 🌐 A2A Protocol Implementation

Your financial assistant now uses **proper A2A (Agent-to-Agent) protocol** with remote agent communication!

### Architecture Overview:

```
┌─────────────────────────────────────────────────────────────┐
│                    Root Coordinator Agent                    │
│              (gemini-2.5-flash-lite)                        │
│         Orchestrates all specialist agents via A2A          │
└────────────────┬────────────────┬───────────────────┬───────┘
                 │                │                   │
                 ▼                ▼                   ▼
    ┌───────────────────┐ ┌──────────────────┐ ┌──────────────────┐
    │ RemoteA2aAgent    │ │ RemoteA2aAgent   │ │ RemoteA2aAgent   │
    │ (Calculator)      │ │ (Watchlist)      │ │ (Search)         │
    └─────────┬─────────┘ └────────┬─────────┘ └────────┬─────────┘
              │                    │                     │
         A2A Protocol         A2A Protocol          A2A Protocol
              │                    │                     │
              ▼                    ▼                     ▼
    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
    │ Calculator      │  │ Watchlist       │  │ Search          │
    │ A2A Server      │  │ A2A Server      │  │ A2A Server      │
    │ Port: 8001      │  │ Port: 8002      │  │ Port: 8003      │
    └─────────────────┘  └─────────────────┘  └─────────────────┘
```

### How A2A Works:

1. **Agent Cards** 📋
   - Each specialist agent publishes a JSON "business card" at `/.well-known/agent-card.json`
   - Contains agent name, description, skills (your Python functions), and endpoints
   - Standard discovery mechanism for A2A protocol

2. **RemoteA2aAgent Proxies** 🔌
   - Client-side proxies that connect to remote A2A servers
   - Root agent uses them like local sub-agents
   - Handle all HTTP communication transparently

3. **Uvicorn Servers** 🚀
   - Each specialist agent runs as a FastAPI/Starlette server
   - Handles A2A protocol requests/responses
   - Can run on different machines (distributed architecture)

4. **Benefits** ✨
   - **Scalability**: Agents can run on separate machines
   - **Isolation**: Each agent has its own process and resources
   - **Interoperability**: Any A2A-compatible agent can discover and use your agents
   - **Proper Error Handling**: Structured communication via HTTP/JSON
   - **Production-Ready**: Standard protocol used in real deployments

### Execution Flow:

```
User Query → Root Agent → RemoteA2aAgent proxy → HTTP Request 
→ A2A Server → Specialist Agent → Tool Execution → Response 
→ HTTP Response → RemoteA2aAgent → Root Agent → User
```

### Testing:

Run the test cells to see A2A communication in action! Each test demonstrates:
- Root agent receiving user query
- Delegating to specialist agents via A2A
- Specialist agents executing tools
- Results flowing back through A2A protocol
- Root agent synthesizing final response

In [12]:
# 🧪 Test 6: Session Memory (A2A Communication) - FIXED!
print("🧪 Testing A2A Communication: Session Memory\n")
print("🔄 Resetting session to start fresh conversation...\n")

# Reset to ensure we start with a clean session
await reset_session()

# These queries will now share the same session and remember context!
await test_a2a_query("Hi! My name is Sam and I'm from Poland.")
await test_a2a_query("What's my name and where am I from?")

🧪 Testing A2A Communication: Session Memory

🔄 Resetting session to start fresh conversation...

✅ Session reset! New session ID: a2a_persistent_session_e0891a89

👤 User: Hi! My name is Sam and I'm from Poland.

🤖 Financial Assistant response:
--------------------------------------------------------------------------------
Hi Sam! It's nice to meet you. How can I help you with your financial needs today?
--------------------------------------------------------------------------------

👤 User: What's my name and where am I from?

🤖 Financial Assistant response:
--------------------------------------------------------------------------------
Hi Sam! It's nice to meet you. How can I help you with your financial needs today?
--------------------------------------------------------------------------------

👤 User: What's my name and where am I from?

🤖 Financial Assistant response:
--------------------------------------------------------------------------------
Your name is Sam and you are 

In [13]:
# 🧪 Test 5: Portfolio Analysis (A2A Communication)
print("🧪 Testing A2A Communication: Portfolio Analysis\n")

await test_a2a_query("Analyze my portfolio: AAPL $10000, MSFT $8000, NVDA $12000. Is this well diversified?")


🧪 Testing A2A Communication: Portfolio Analysis


👤 User: Analyze my portfolio: AAPL $10000, MSFT $8000, NVDA $12000. Is this well diversified?

🤖 Financial Assistant response:
--------------------------------------------------------------------------------


/Users/lujain/Library/Python/3.11/lib/python/site-packages/google/adk/agents/remote_a2a_agent.py:379: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)
/Users/lujain/Library/Python/3.11/lib/python/site-packages/google/adk/agents/remote_a2a_agent.py:379: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedbac

The total value of your portfolio is $30,000. Your current allocation is:

*   AAPL: 33.33% ($10,000)
*   MSFT: 26.67% ($8,000)
*   NVDA: 40% ($12,000)

Regarding diversification, your portfolio is heavily concentrated in NVDA (40%) and AAPL (33.33%). While these are significant companies, having such a large portion of your assets in just a few stocks, especially in the tech sector, can be risky. A well-diversified portfolio typically spreads investments across various asset classes (stocks, bonds, real estate, etc.), industries, and geographies to mitigate risk.

To improve diversification, you might consider reducing your allocation to the largest holdings and exploring investments in other sectors or asset types that have lower correlations with your current investments.
--------------------------------------------------------------------------------


/Users/lujain/Library/Python/3.11/lib/python/site-packages/google/adk/a2a/converters/event_converter.py:239: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/Users/lujain/Library/Python/3.11/lib/python/site-packages/google/adk/a2a/converters/event_converter.py:309: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback

In [14]:
# 🧪 Test 1: Search + Calculate + Watchlist (A2A Communication with Session Memory)
print("🧪 Testing A2A Communication: Search → Calculate → Watchlist\n")

# Create a shared session for this conversation
shared_session_id = f"a2a_session_{uuid.uuid4().hex[:8]}"
print(f"📝 Using session ID: {shared_session_id}\n")

# Create the session ONCE before all queries
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=shared_session_id
)
print(f"✅ Session created successfully!\n")

async def test_with_memory(user_query: str, session_id: str):
    """Test query with session memory."""
    test_content = types.Content(parts=[types.Part(text=user_query)])
    
    print(f"\n👤 User: {user_query}")
    print(f"🤖 Financial Assistant response:")
    print("-" * 80)
    
    # Now runner can use the existing session
    async for event in runner.run_async(
        user_id=USER_ID, session_id=session_id, new_message=test_content
    ):
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)
    
    print("-" * 80)

# Query 1: Search for Tesla info
await test_with_memory("Find Tesla's current stock price and EPS.", shared_session_id)

# Query 2: Calculate P/E ratio (should remember Tesla from Query 1)
await test_with_memory("Now calculate Tesla's P/E ratio using that information.", shared_session_id)

# Query 3: Check watchlist
await test_with_memory("What's in my watchlist?", shared_session_id)


🧪 Testing A2A Communication: Search → Calculate → Watchlist

📝 Using session ID: a2a_session_6be2b60f

✅ Session created successfully!


👤 User: Find Tesla's current stock price and EPS.
🤖 Financial Assistant response:
--------------------------------------------------------------------------------


Here's the information on Tesla's (TSLA) stock price and EPS:

*   **Stock Price:** As of November 29, 2025, at 8:39:20 AM UTC, the stock price is $430.17.
*   **Earnings Per Share (EPS):**
    *   The EPS for the twelve months ending September 30, 2025, was $1.45.
    *   Tesla's basic earnings per share (Basic EPS) for the trailing twelve months (TTM) ended in September 2025 was $1.64.
    *   The EPS for the quarter ending September 30, 2025, was $0.39.


--------------------------------------------------------------------------------

👤 User: Now calculate Tesla's P/E ratio using that information.
🤖 Financial Assistant response:
--------------------------------------------------------------------------------


The Price-to-Earnings (P/E) ratio for Tesla is calculated as follows:

P/E Ratio = Stock Price / Earnings Per Share (EPS)
P/E Ratio = $430.17 / $1.64
P/E Ratio = 262.3

Tesla's P/E ratio is 262.3. This indicates that investors are willing to pay $262.30 for every $1.00 of Tesla's earnings. A high P/E ratio can suggest that investors expect higher earnings growth in the future, or that the stock may be overvalued.
--------------------------------------------------------------------------------

👤 User: What's in my watchlist?
🤖 Financial Assistant response:
--------------------------------------------------------------------------------


There are no stocks in your watchlist currently.
--------------------------------------------------------------------------------


In [15]:
# 🧪 Test 3: Financial Calculations (A2A Communication)
print("🧪 Testing A2A Communication: Financial Calculations\n")

await test_a2a_query("If a stock trades at $200 with EPS of $10, what's the P/E ratio? Is this good?")


🧪 Testing A2A Communication: Financial Calculations


👤 User: If a stock trades at $200 with EPS of $10, what's the P/E ratio? Is this good?

🤖 Financial Assistant response:
--------------------------------------------------------------------------------


The P/E ratio for this stock is 20.

Here's the calculation:
P/E Ratio = Stock Price / Earnings Per Share (EPS)
P/E Ratio = $200 / $10 = 20

Whether a P/E ratio of 20 is "good" depends on several factors:

*   **Industry Average:** P/E ratios vary significantly by industry. A P/E of 20 might be high for a mature, slow-growing industry but could be considered reasonable or even low for a high-growth sector like technology.
*   **Company Growth Prospects:** If the company is expected to grow its earnings rapidly, investors may be willing to pay a higher P/E ratio.
*   **Overall Market Conditions:** In a bull market, P/E ratios tend to be higher across the board.

A P/E of 20 is generally considered moderate. It suggests that investors are willing to pay $20 for every $1 of the company's annual earnings. It's important to compare this P/E ratio to the company's historical P/E ratios, its competitors' P/E ratios, and the overall market P/E to make a more informed judgment.
----------------

In [16]:
# 🧪 Test 4: Google Search (A2A Communication)
print("🧪 Testing A2A Communication: Market Research\n")

await test_a2a_query("What's the latest news about NVIDIA stock and their AI chip business?")


🧪 Testing A2A Communication: Market Research


👤 User: What's the latest news about NVIDIA stock and their AI chip business?

🤖 Financial Assistant response:
--------------------------------------------------------------------------------


Okay, Sam from Poland! I can help you with that.

Here's the latest news regarding NVIDIA stock and their AI chip business:


Here's the latest news on NVIDIA and its AI chip business:

*   **Stock Performance:** Despite strong earnings reports, NVIDIA's stock has faced some challenges. There was a recent drop in stock value, with some analysts attributing it to concerns about overspending on AI infrastructure by tech giants.
*   **Strong Earnings:** NVIDIA's recent quarterly results exceeded expectations, with a significant increase in revenue and earnings per share. The company's data center business, crucial for AI, saw substantial growth.
*   **AI Chip Market:** NVIDIA remains a leader in the AI chip market. However, there's growing competition, particularly from Google, which is promoting its Tensor Processing Units (TPUs) as alternatives to NVIDIA's GPUs.
*   **Competition from Google and Meta:** Google is aggressively pitching its TPUs to major clients, including Meta. This move

In [ ]:
# 🧪 Test 2: Watchlist Management (A2A Communication)
print("🧪 Testing A2A Communication: Watchlist Management\n")
await test_a2a_query("add apple to my watchlist.")

await test_a2a_query("Add TSLA to my watchlist. I'm interested in their EV market dominance.")
await test_a2a_query("Show me my complete watchlist.")
#await test_a2a_query("ADD AAPL from my watchlist.")


🧪 Testing A2A Communication: Watchlist Management


👤 User: add apple to my watchlist.

🤖 Financial Assistant response:
--------------------------------------------------------------------------------


I've added Apple to your watchlist. Any particular reason you're interested in them? Knowing the reason can help you track your investment goals.
--------------------------------------------------------------------------------

👤 User: Show me my complete watchlist.

🤖 Financial Assistant response:
--------------------------------------------------------------------------------


Here is your watchlist:

*   APPLE
--------------------------------------------------------------------------------
